# `BERT`
**`BERT`** (от англ. <i>Bidirectional Encoder Representations from Transformers, «двунаправленная нейронная сеть-кодировщик»</i>) — нейронная сеть для создания модели языка. Её разработали в компании $Google$, чтобы повысить релевантность результатов поиска. Этот алгоритм понимает контекст запросов, а не просто анализирует фразы. Для машинного обучения она ценна тем, что помогает строить векторные представления. Причём в анализе текстов применяют уже предобученную на большом корпусе модель. Такие предобученные версии BERT годятся для работы с текстами на 104 языках мира, включая русский.
<br>**`BERT`** — это результат эволюции модели `word2vec`. В ходе её развития были придуманы и другие модели: `FastText` (англ. «быстрый текст»), `GloVe` (англ. <i>Global Vectors for Word Representation, «глобальные векторы для языкового представления»</i>), `ELMO` (англ. <i>Embeddings from Language Models, «вложения языковых моделей»</i>) и `GPT` (англ. <i>Generative Pre-Training Transformer, «предобученный трансформер для генерации»</i>). Сейчас самые точные — это `BERT` и `GPT-3`, которого нет в открытом доступе.
<br>`BERT` учитывает контекст не только соседних слов, но и более дальних родственников. Работает так:
- На входе модель получает, например, такую фразу:
<br>«Красный клюв тупика [MASK] на голубом [MASK]», где MASK (англ. «маска») это неизвестные слова, будто закрытые маской. Модель должна угадать эти спрятанные слова.
- Модель обучается определять, связаны ли в предложении слова между собой. У нас были скрыты такие слова:
<br>«мелькнул» и «небе». Модель должна понять, что одно слово — продолжение другого. Скажем, если вместо «мелькнул» спрятать слово «прополз», то связи модель не найдёт.
# `RuBERT` и предобработка
Построить векторы текстов нам поможет предобученная на русских текстах модель RuBERT.
# Задача
Перед вами большой датасет с твитами. Нужно научиться определять, какие твиты негативной тональности, а какие — позитивной. Чтобы решить эту задачу, из открытого репозитория [`DeepPavlov`](https://docs.deeppavlov.ai/en/master/features/models/bert.html) возьмём модель `RuBERT`, обученную на разговорном русскоязычном корпусе. 
<br>Решим эту задачу на `PyTorch` (англ. <i>«факел для Python»</i>). Hужна для работы с моделями типа `BERT`. Они находятся в библиотеке `transformers` (англ. <i>«трансформеры»</i>). Импортируем их:

In [2]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install evaluate

  Attempting uninstall: fsspec
    Found existing installation: fsspec 0.6.2
    Uninstalling fsspec-0.6.2:
      Successfully uninstalled fsspec-0.6.2
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.42.1
    Uninstalling tqdm-4.42.1:
      Successfully uninstalled tqdm-4.42.1
Note: you may need to restart the kernel to use updated packages.


ERROR: conda-build 3.18.11 requires conda, which is not installed.
ERROR: datasets 2.13.2 has requirement dill<0.3.7,>=0.3.0, but you'll have dill 0.3.7 which is incompatible.
ERROR: responses 0.18.0 has requirement urllib3>=1.25.10, but you'll have urllib3 1.25.8 which is incompatible.


In [1]:
import torch
import transformers

Прежде чем перевести тексты в векторы, подготовим их. У RuBERT есть собственный токенизатор. Это инструмент, который разбивает и преобразует исходные тексты в список токенов, которые есть, например, в словаре RuBERT. Лемматизация не требуется.
<br>Начинаем предобработку текстов:
1. Инициализируем токенизатор как объект класса `BertTokenizer()`. Передадим ему аргумент `vocab_file` — это файл со словарём, на котором обучалась модель. Он может быть, например, в текстовом формате ($txt$).

In [3]:
tokenizer = transformers.BertTokenizer(
    vocab_file='tweets_lemm_test.csv')#'/datasets/ds_bert/vocab.txt')

2. Преобразуем текст в номера токенов из словаря методом `encode()` (англ. <i>«закодировать»</i>):

In [7]:
tokenizer.encode('Очень удобно использовать уже готовый трансформатор текста', add_special_tokens=True)


яндекс на*бывает, херь не работает, падает в ошибку, типа не тензор передан, они *ёбки дикие

Для корректной работы модели мы указали аргумент `add_special_tokens` (англ. <i>«добавить специальные токены»</i>), равный $True$. Это значит, что к любому преобразуемому тексту добавляется токен начала (101) и токен конца текста (102). 
3. Применим метод `padding` (англ. <i>«отступ»</i>), чтобы после токенизации длины исходных текстов в корпусе были равными. Только при таком условии будет работать модель `BERT`. Пусть стандартной длиной вектора $n$ будет длина наибольшего во всём датасете вектора. Остальные векторы дополним нулями:

In [9]:
vector = tokenizer.encode('Очень удобно использовать уже готовый трансформатор текста', add_special_tokens=True)
n = 280
# англ. вектор с отступами
padded = np.array(vector + [0]*(n - len(vector)))
print(padded)

Теперь поясним модели, что нули не несут значимой информации. Это нужно для компоненты модели, которая называется «внимание» (англ. <i>attention</i>). Отбросим эти токены и «создадим маску» для действительно важных токенов, то есть укажем нулевые и не нулевые значения:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
print(attention_mask.shape)

# Эмбеддинги RuBERT
Инициализируем конфигурацию `BertConfig` (англ. <i>Bert Configuration</i>). В качестве аргумента передадим ей JSON-файл с описанием настроек модели. `JSON` (англ. <i>JavaScript Object Notation, «объектная запись JavaScript»</i>) — это организованный по ключам поток цифр, букв, двоеточий и фигурных скобок, который возвращает сервер при запросе. 
Затем инициализируем саму модель класса `BertModel`. Передадим ей файл с предобученной моделью и конфигурацией:

In [ ]:
config = transformers.BertConfig.from_json_file(
    '/datasets/ds_bert/bert_config.json')
model = transformers.BertModel.from_pretrained(
    '/datasets/ds_bert/rubert_model.bin', config=config)

Начнём преобразование текстов в эмбеддинги. Это может занять несколько минут, поэтому подключим библиотеку `tqdm` (араб. <i>taqadum, تقدّم, «прогресс»</i>). Она нужна, чтобы наглядно показать индикатор прогресса. В Jupyter применим функцию `notebook()` из этой библиотеки: 

In [10]:
from tqdm import notebook

Эмбеддинги модель `BERT` создаёт батчами. Чтобы хватило оперативной памяти, сделаем размер батча небольшим:

In [ ]:
batch_size = 100 

Сделаем цикл по батчам. Отображать прогресс будет функция `notebook()`
<br>01) Преобразуем данные в формат тензоров (англ. <i>tensor</i>) — многомерных векторов в библиотеке `torch`. Тип данных `LongTensor` (англ. <i>«длинный тензор»</i>) хранит числа в «длинном формате», то есть выделяет на каждое число 64 бита.:
<br>02) Чтобы получить эмбеддинги для батча, передадим модели данные и маску:
<br>03) Для ускорения вычисления функцией `no_grad()` (англ. <i>no gradient, «нет градиента»</i>) в библиотеке torch укажем, что градиенты не нужны: модель `BERT` обучать не будем.
<br>04) Из полученного тензора извлечём нужные элементы и добавим в список всех эмбеддингов:


In [ ]:
# сделаем пустой список для хранения эмбеддингов твитов
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    #01) преобразуем данные
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)])
    # 02)преобразуем маску
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
#     02) получить эмбеддинги для батча
    batch_embeddings = model(batch, attention_mask=attention_mask_batch)
#     03) no_grad()
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    #04) преобразуем элементы методом numpy() к типу numpy.array
    embeddings.append(batch_embeddings[0][:,0,:].numpy()) 

Соберём все эмбеддинги в матрицу признаков вызовом функции `concatenate()`:

In [ ]:
features = np.concatenate(embeddings)